In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import date, datetime, timedelta
import warnings
import pytz
import streamlit.components.v1 as components


from tzlocal import get_localzone  # pip install tzlocal
import time
import chess
import chess.pgn
import io
import pprint
import requests
warnings.filterwarnings('ignore')
import chess_com_download
from streamlit_javascript import st_javascript
from pytz import timezone

from chess_com_download import test_function_test
from chess_com_download import opening_database
from chess_com_download import download_data_chess_com
from chess_com_download import pre_analysis_chessgame
from chess_com_download import analyze_chess_games
from chess_com_download import prepare_game_data
from chess_com_download import insert_to_supabase


from supabase import create_client, Client
from datetime import datetime
import pytz


import streamlit.runtime.app_session as app_session

if not hasattr(app_session.AppSession, "_scriptrunner"):
    app_session.AppSession._scriptrunner = None



In [ ]:
# Try to detect local timezone and map it to a valid pytz zone
try:
    import tzlocal  # pip install tzlocal
    local_tz_name = tzlocal.get_localzone_name()  # e.g., 'Asia/Jakarta'
except:
    local_tz_name = "UTC"  # fallback

# If session_state doesn't have timezone, set it to detected
if "user_timezone" not in st.session_state:
    st.session_state.user_timezone = local_tz_name

# Get current time in user's timezone
user_timezone = pytz.timezone(st.session_state.user_timezone)



df_chess_game = download_data_chess_com('jay_fh',7,8,2025)
display (df_chess_game.head() )
# st.set_page_config(
#     page_title="Chess.com Game Analyzer",
#     page_icon="♟️",
#     layout="wide",
#     initial_sidebar_state="expanded"
# )

# user_timezone = pytz.timezone(st.session_state.user_timezone)
# user_timezone=timezone('Asia/Jakarta')
df = pre_analysis_chessgame(df_chess_game, 'jay_fh',user_timezone)
display (df.head())

timecontrol_counts = df['TimeControl'].value_counts()
    # Get the TimeControl values that appear more than 20 times
popular_timecontrols = timecontrol_counts[timecontrol_counts > 50].index
print (popular_timecontrols)
df_final = df[df['TimeControl'].isin(popular_timecontrols)]
display (df_final.head())
df['time_of_day'].value_counts() 
# df_chess_game = download_data_chess_com(chesscom_user_id,3,6,2025)
    
    # games_data = prepare_game_data(df_chess_game)
    # total_inserted = insert_to_supabase(games_data)
    # print(f"\nSuccessfully inserted {total_inserted} out of {len(df_chess_game)} games")
# df = pre_analysis_chessgame(df_chess_game, chesscom_user_id,user_timezone)

In [ ]:
top5_opening = df['simplified_opening'].value_counts().head(5).index

df_opening = df[df['simplified_opening'].isin(top5_opening)].copy()
# df_opening


df_opening["is_win"] = (df_opening["game_result_user"] == "win").astype(int)
df_opening['is_win']

# ---- Calculate cumulative win rate per opening over time ----
# df_opening = df.sort_values("created_datetime")

df_opening["games_count"] = df_opening.groupby("simplified_opening").cumcount() + 1
df_opening["cum_wins"] = df_opening.groupby("simplified_opening")["is_win"].cumsum()
df_opening["cum_wins"]
# df_opening["cum_win_rate"] = df_opening["cum_wins"] / df["games_count"] * 100

fig = px.line(
    df,
    x="created_datetime",
    y="cum_win_rate",
    color="simplified_opening",
    markers=True,
    title="Win Rate Over Time per Opening"
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Cumulative Win Rate (%)",
    legend_title="Opening"
)

fig.show()

# ---- Streamlit Render ----
# st.title("♟️ Opening Performance Over Time")

# st.plotly_chart(fig, use_container_width=True)


In [13]:
# data_a = "https://github.com/Jayfetra/streamlit_goodreads_app/blob/master/chess-openings-master/a.tsv"
data_a = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/a.tsv"
# data_b = "https://github.com/Jayfetra/streamlit_goodreads_app/blob/master/chess-openings-master/b.tsv"
# data_c = "https://github.com/Jayfetra/streamlit_goodreads_app/blob/master/chess-openings-master/c.tsv"
# data_d = "https://github.com/Jayfetra/streamlit_goodreads_app/blob/master/chess-openings-master/d.tsv"
# data_e = "https://github.com/Jayfetra/streamlit_goodreads_app/blob/master/chess-openings-master/e.tsv"


print (data_a)

df_a = pd.read_csv(data_a, sep="\t")
# df_b = pd.read_csv(data_b, sep="\t")
# df_c = pd.read_csv(data_c, sep="\t")
# df_d = pd.read_csv(data_d, sep="\t")
# df_e = pd.read_csv(data_e, sep="\t")

df_a

# Combine the DataFrames
# chess_opening_database = pd.concat([df_a, df_b, df_c, df_d, df_e], ignore_index=True)
# chess_opening_database

https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/a.tsv


,eco,name,pgn
0,A00,Amar Opening,1. Nh3
1,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4
2,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...
3,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6
4,A00,Anderssen's Opening,1. a3
...,...,...,...
723,A96,"Dutch Defense: Classical Variation, Huisl Vari...",1. d4 f5 2. c4 Nf6 3. g3 e6 4. Bg2 Be7 5. Nf3 ...
724,A97,"Dutch Defense: Classical Variation, Ilyin-Zhen...",1. d4 f5 2. c4 Nf6 3. g3 e6 4. Bg2 Be7 5. Nf3 ...
725,A97,"Dutch Defense: Classical Variation, Ilyin-Zhen...",1. d4 f5 2. c4 Nf6 3. g3 e6 4. Bg2 Be7 5. Nf3 ...
726,A98,"Dutch Defense: Classical Variation, Ilyin-Zhen...",1. d4 f5 2. c4 Nf6 3. g3 e6 4. Bg2 Be7 5. Nf3 ...


In [9]:
import os

chess_opening_dir = f'/Users/jayson.fetra/Documents/c_root/hobby/chess-openings-master'
os.chdir(chess_opening_dir)

cwd = os.getcwd()
dir_list = os.listdir(cwd)
print (dir_list)

# Read the TSV files into DataFrames
df_a = pd.read_csv('a.tsv', sep='\t')
df_b = pd.read_csv('b.tsv', sep='\t')
df_c = pd.read_csv('c.tsv', sep='\t')
df_d = pd.read_csv('d.tsv', sep='\t')
df_e = pd.read_csv('e.tsv', sep='\t')

chess_opening_database = pd.concat([df_a, df_b, df_c, df_d, df_e], ignore_index=True)
chess_opening_database

['COPYING.txt', 'jay_fh_historical_data.txt', '.DS_Store', 'bin', 'Makefile', 'README.md', 'jay_fh_historical_data.csv', '.gitignore', 'CONTRIBUTING.md', '.github', 'e.tsv', 'd.tsv', 'c.tsv', 'b.tsv', 'a.tsv']


,eco,name,pgn
0,A00,Amar Opening,1. Nh3
1,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4
2,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...
3,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6
4,A00,Anderssen's Opening,1. a3
...,...,...,...
3512,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
3513,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
3514,E98,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
3515,E99,"King's Indian Defense: Orthodox Variation, Cla...",1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...


In [14]:

def extract_moves_from_pgn(pgn_string):
    pgn_io = io.StringIO(pgn_string)
    game = chess.pgn.read_game(pgn_io)
    board = game.board()
    moves = []
    for move in game.mainline_moves():
        moves.append(board.san(move))
        board.push(move)
    return moves

def opening_database():

    # Read the TSV files into DataFrames
    # a_df = pd.read_csv('a.tsv', sep='\t')
    # b_df = pd.read_csv('b.tsv', sep='\t')
    # c_df = pd.read_csv('c.tsv', sep='\t')
    # d_df = pd.read_csv('d.tsv', sep='\t')
    # e_df = pd.read_csv('e.tsv', sep='\t')
    

    data_a = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/a.tsv"
    data_b = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/b.tsv"
    data_c = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/c.tsv"
    data_d = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/d.tsv"
    data_e = "https://raw.githubusercontent.com/Jayfetra/streamlit_goodreads_app/refs/heads/master/chess-openings-master/e.tsv"

    df_a = pd.read_csv(data_a, sep="\t")
    df_b = pd.read_csv(data_b, sep="\t")
    df_c = pd.read_csv(data_c, sep="\t")
    df_d = pd.read_csv(data_d, sep="\t")
    df_e = pd.read_csv(data_e, sep="\t")

    # Combine the DataFrames
    chess_opening_database = pd.concat([df_a, df_b, df_c, df_d, df_e], ignore_index=True)
    chess_moves = []
    # Iterate over the rows of the combined DataFrame
    for index, row in chess_opening_database.iterrows():
        pgn_string = row['pgn']
        moves = extract_moves_from_pgn(pgn_string)
        chess_moves.append(moves)

        # Add the new column of opening_moves name to the DataFrame
    chess_opening_database['opening_moves'] = chess_moves

    # chess_database_opening_df.head()
    return chess_opening_database

chess_database_opening_df = opening_database()
display (chess_database_opening_df.head()  )

,eco,name,pgn,opening_moves
0,A00,Amar Opening,1. Nh3,[Nh3]
1,A00,Amar Opening: Paris Gambit,1. Nh3 d5 2. g3 e5 3. f4,"[Nh3, d5, g3, e5, f4]"
2,A00,"Amar Opening: Paris Gambit, Gent Gambit",1. Nh3 d5 2. g3 e5 3. f4 Bxh3 4. Bxh3 exf4 5. ...,"[Nh3, d5, g3, e5, f4, Bxh3, Bxh3, exf4, O-O, f..."
3,A00,Amsterdam Attack,1. e3 e5 2. c4 d6 3. Nc3 Nc6 4. b3 Nf6,"[e3, e5, c4, d6, Nc3, Nc6, b3, Nf6]"
4,A00,Anderssen's Opening,1. a3,[a3]
